In [ ]:
#!pip install numpy nibabel glob2 scipy tensorflow matplotlib firebase-admin

In [ ]:
import numpy as np
import nibabel as nib
import glob
from scipy.ndimage import zoom
from io import BytesIO
import os 
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv3D, MaxPooling3D, UpSampling3D, concatenate
from sklearn.metrics import mean_squared_error
import random
import matplotlib.pyplot as plt
import firebase_admin
from firebase_admin import credentials, storage
import tempfile

In [ ]:
#Train and Labels size + 1 for the loops iteration 
train_size = 126+1 #Max 304
test_size = 13+1 #Max 140

#Common shape for resizing
common_shape = (512, 512, 64)

#Common shape for downsizing
downsized_shape = (256, 256, 32)

In [ ]:
cred = credentials.Certificate('serviceAccountKey.json')
firebase_admin.initialize_app(cred, {'storageBucket': 'api-ml-17a83.appspot.com'})

# Get a reference to the default bucket
bucket = storage.bucket()

# Create the array with folder names
folders = ["Label", "Test", "Train"]

In [ ]:
# List blobs in the specific folder and print their names
blobs_label = bucket.list_blobs(prefix=folders[0]) 
blobs_test = bucket.list_blobs(prefix=folders[1])
blobs_train = bucket.list_blobs(prefix=folders[2])

In [ ]:
#Functino to download images files from firebase storage
def load_data(blobs, max_size):
    
    def blob_to_list():
        arr = []
        counter = 0 
        for blob in blobs:
            if counter == 0:
                print("Skipping the first iteration")
                counter += 1
                continue  # Skip the first iteration
            if counter <= max_size:
                image_bytes = blob.download_as_bytes()
                print(f"Processing and Downloaded blob: {blob.name}")
                image_io = BytesIO(image_bytes)
                counter += 1
                arr.append(image_io)
            else:
                break

        return arr
    
    data_list = blob_to_list()

    #Convert Download bytes to medical image
    def bytes_to_medical_image(bytes_list):
        medical_images = []
        temp_files = []  # Store temporary file objects in this list
        for byte_io in bytes_list:
            try:
                # Save the BytesIO data to a temporary file
                temp_file = tempfile.NamedTemporaryFile(suffix='.nii.gz', delete=False)
                temp_file.write(byte_io.getvalue())
                temp_file.flush()
                temp_files.append(temp_file)  # Add the temporary file object to the list

                # Load the medical image from the temporary file using nibabel
                image = nib.load(temp_file.name)
                medical_images.append(image)
            except nib.filebasedimages.ImageFileError as e:
                print(f"Error loading image: {e}")

        # Close and clean up all the temporary files after processing all the images
        for temp_file in temp_files:
            temp_file.close()
            # Uncomment the next line if you want to delete the temporary file after use
            # os.unlink(temp_file.name)

            return medical_images

    medical_data = bytes_to_medical_image(data_list)

    return medical_data

In [ ]:
#Loading train images
train_images = load_data(blobs_train ,train_size)

In [ ]:
#Loading label images
labels = load_data(blobs_label ,train_size)

In [ ]:
#Convert images so could use them in modle
common_train_images = []
for nifti_image in train_images:
    # Convert the Nifti1Image to a numpy array
    image_data = nifti_image.get_fdata()

    # Resize the image data to the common shape
    resized_image = zoom(image_data, common_shape / np.array(image_data.shape), order=1)
    common_train_images.append(resized_image)

print("All train images resized to the common shape.")

In [ ]:
#Convert images so could use them in modle
common_labels = []
for nifti_label in train_images:
    # Convert the Nifti1Image to a numpy array
    label_data = nifti_label.get_fdata()

    # Resize the image data to the common shape
    zoom_factors = (common_shape[0] / label_data.shape[0], common_shape[1] / label_data.shape[1], common_shape[2] / label_data.shape[2])
    resized_label = zoom(label_data, zoom_factors, order=1)
    common_labels.append(resized_label)

print("All train images resized to the common shape.")

common_labels = np.array(common_labels)

In [ ]:
# Compute the mean and standard deviation of the training images
train_mean = np.mean(common_train_images)
train_std = np.std(common_train_images)

# Normalize the images
train_images_normalized = (common_train_images - train_mean) / train_std

# Optionally, you can clip the pixel values to a specific range (e.g., [0, 1])
train_images_normalized = np.clip(train_images_normalized, 0, 1)

In [ ]:
# Create an ImageDataGenerator with lighter augmentation options
data_augmentation = ImageDataGenerator(
    rotation_range=5,           # Reduce rotation range to 5 degrees
    width_shift_range=0.05,     # Reduce horizontal shift range
    height_shift_range=0.05,    # Reduce vertical shift range
    shear_range=0.1,            # Reduce shear transformations
    zoom_range=0.1,             # Reduce zoom-in/out range
    horizontal_flip=True,       # Keep random horizontal flips
    vertical_flip=True,         # Keep random vertical flips
    fill_mode='reflect'         # Fill mode for newly created pixels
)

In [ ]:
# Function to apply data augmentation to a set of images and labels
def augment_images(images, labels):
    augmented_images = []
    augmented_labels = []
    for image, label in zip(images, labels):
        # Expand the image and label dimensions to match the required input shape for ImageDataGenerator
        image = np.expand_dims(image, axis=0)
        label = np.expand_dims(label, axis=0)

        # Apply data augmentation to both image and label
        augmented_image = data_augmentation.flow(image, batch_size=1, shuffle=False)
        augmented_label = data_augmentation.flow(label, batch_size=1, shuffle=False)

        augmented_images.append(next(augmented_image)[0])
        augmented_labels.append(next(augmented_label)[0])

    return np.array(augmented_images), np.array(augmented_labels)

# Apply data augmentation to the normalized training images and labels with lighter settings
augmented_train_images, augmented_train_labels = augment_images(train_images_normalized, common_labels)

In [ ]:
# Function to build a light 3D U-Net model
def build_light_3d_unet(input_shape):
    # Define the input layer of the model with the specified input shape
    inputs = Input(input_shape)

    # Encoder
    # Add a 3D convolutional layer with 32 filters, kernel size (3, 3, 3), ReLU activation, and padding to maintain the spatial dimensions
    conv1 = Conv3D(32, (3, 3, 3), activation='relu', padding='same')(inputs)
    
    # Add a 3D max pooling layer with downsampling factor (2, 2, 2) to reduce spatial dimensions
    pool1 = MaxPooling3D((2, 2, 2))(conv1)

    # Decoder
    # Add a 3D upsampling layer with upsampling factor (2, 2, 2) to increase spatial dimensions
    up8 = UpSampling3D((2, 2, 2))(pool1)

    # Output layer
    # Add a 3D convolutional layer with 1 filter, kernel size (1, 1, 1), sigmoid activation, to produce the final output
    outputs = Conv3D(1, (1, 1, 1), activation='sigmoid')(up8)

    # Create the Keras model with input and output layers
    model = Model(inputs=inputs, outputs=outputs)

    return model

In [ ]:
# Function to downsize images and labels
def resize_data(images, labels, target_shape):
    resized_images = []
    resized_labels = []
    for image, label in zip(images, labels):
        # Resize the image and label to the target shape
        resized_image = zoom(image, target_shape / np.array(image.shape), order=1)
        resized_label = zoom(label, target_shape / np.array(label.shape), order=1)

        # Append the resized image and label to the list
        resized_images.append(resized_image)
        resized_labels.append(resized_label)

    return np.array(resized_images), np.array(resized_labels)

# Downsize the images and labels
downsized_train_images, downsized_train_labels = resize_data(common_train_images, common_labels, downsized_shape)

# Normalize the downsized images
downsized_train_images_normalized = (downsized_train_images - train_mean) / train_std

# Apply data augmentation to the normalized downsized training images and labels with lighter settings
augmented_downsized_train_images, augmented_downsized_train_labels = augment_images(downsized_train_images_normalized, downsized_train_labels)

In [ ]:
# Assuming augmented_downsized_train_images is a numpy array with shape (num_samples, 256, 256, 32)
input_shape = augmented_downsized_train_images.shape[1:]  # Shape of the augmented training images

# Add the batch size and channel dimension to the input shape
input_shape = tuple(list(input_shape) + [1])  # Add 1 for a single channel

light_model = build_light_3d_unet(input_shape)
light_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the light model with the downsized and augmented data
history = light_model.fit(augmented_downsized_train_images, augmented_downsized_train_labels, batch_size=5, epochs=10, validation_split=0.2)

In [ ]:
# Print training accuracy and loss for each epoch
train_acc = history.history['accuracy']
train_loss = history.history['loss']
val_acc = history.history['val_accuracy']  # Add this line to get validation accuracy
val_loss = history.history['val_loss']

#Print each epoch accuracy and loss
for epoch, (acc, loss) in enumerate(zip(train_acc, train_loss), 1):
    print(f"Epoch {epoch}: Training Accuracy = {acc:.4f}, Training Loss = {loss:.4f}")

In [ ]:
# Plot the training and validation loss
plt.figure(figsize=(10, 5))
plt.plot(range(1, len(train_loss) + 1), train_loss, label='Training Loss')
plt.plot(range(1, len(val_loss) + 1), val_loss, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Training and Validation Loss')
plt.show()

# Plot the training and validation accuracy
plt.figure(figsize=(10, 5))
plt.plot(range(1, len(train_acc) + 1), train_acc, label='Training Accuracy')
plt.plot(range(1, len(val_acc) + 1), val_acc, label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Training and Validation Accuracy')
plt.show()

In [ ]:
# Assuming you have the ground truth labels for the augmented downsized training data
# and you have obtained the predictions from the trained model
train_labels_ground_truth = augmented_downsized_train_labels
train_predictions = light_model.predict(augmented_downsized_train_images)

# Flatten the 3D arrays to 1D arrays
flattened_labels_ground_truth = train_labels_ground_truth.ravel()
flattened_predictions = train_predictions.ravel()

# Calculate the Mean Squared Error (MSE)
mse = mean_squared_error(flattened_labels_ground_truth, flattened_predictions)

print("Mean Squared Error:", mse)

In [ ]:
test_images = load_data(blobs_test ,test_size)

In [ ]:
common_test_images = []
for nifti_image in test_images:
    # Convert the Nifti1Image to a numpy array
    image_data = nifti_image.get_fdata()

    # Resize the image data to the common shape
    resized_image = zoom(image_data, common_shape / np.array(image_data.shape), order=1)
    common_test_images.append(resized_image)

print("All test images resized to the common shape.")

In [ ]:
# Function to resize and augment test data
def resize_and_augment_test_data(images, target_shape):
    resized_images = []
    for image in images:
        # Resize the image to the target shape
        resized_image = zoom(image, target_shape / np.array(image.shape), order=1)
        resized_images.append(resized_image)

    resized_images = np.array(resized_images)

    # Normalize the test images using the mean and standard deviation computed from the training data
    normalized_images = (resized_images - train_mean) / train_std

    # Apply data augmentation to the normalized test images with lighter settings
    augmented_images = []
    for image in normalized_images:
        # Expand the image dimensions to match the required input shape for ImageDataGenerator
        image = np.expand_dims(image, axis=0)

        # Apply data augmentation to the image
        augmented_image = data_augmentation.flow(image, batch_size=1, shuffle=False)

        augmented_images.append(next(augmented_image)[0])

    return np.array(augmented_images)

In [ ]:
# Assuming you have already loaded the test images into the 'test_images' array
# and set the 'target_shape' for resizing
augmented_test_images = resize_and_augment_test_data(common_test_images, downsized_shape)

# Make predictions on the augmented test images using your trained model
predictions = light_model.predict(augmented_test_images)

In [ ]:
# Get the indices of 3 random test images
random_indices = random.sample(range(len(common_test_images)), 3)

# Loop through the randomly selected test images and their predictions
for i in random_indices:
    test_image = test_images[i].get_fdata()  # Get the data array of the Nifti1Image
    prediction = predictions[i]

    # Create a plot with two subplots
    fig, axs = plt.subplots(1, 2, figsize=(10, 5))
    axs[0].imshow(test_image[:, :, 0], cmap='gray')  # Assuming you want to display the first slice (leftmost)
    axs[0].set_title('Test Image')

    axs[1].imshow(prediction[:, :, 0], cmap='gray')  # Assuming you want to display the predicted label of the first slice
    axs[1].set_title('Predicted Tumor Region')

    plt.show()